## Setup

In [1]:
from structure import PhotoInfo, EXIFImageMetadata

In [2]:
import os
import openai
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

os.environ['OPENAI_API_KEY'] = config['openai_api_key']

In [3]:
# make a list of image paths in images/stock. Can be .jpg or .png or .jpeg
import os

PHOTOS_DIR = 'images/stock'

image_paths = []
for filename in os.listdir(PHOTOS_DIR):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        image_paths.append(os.path.join(PHOTOS_DIR, filename))
    else:
        continue

print(f"{len(image_paths)} images ready to process")

21 images ready to process


In [4]:
from database import create_table

if os.path.exists("photo_database.db") == False:
    create_table()

In [5]:
import qdrant_client
from llama_index.vector_stores import QdrantVectorStore
from llama_index.storage import StorageContext
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="qdrant_gemini")
vector_store = QdrantVectorStore(client=client, collection_name="collection")

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

# Load the vector store index
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

## Injestion Pipeline

In [6]:
# Image -> Metadata and Tags and is_screenshot -> PhotoInfo
from image_utils import process_image, process_image_list
import asyncio

# photo_info = asyncio.run(process_image(image_paths[0]))
photo_info = process_image_list(image_paths, injestion = False, get_tags = False)
photo_info 

Photo images/stock/image3.jpg already exists in the database. Using cache values...
Photo images/stock/phone_photo (2).jpg already exists in the database. Using cache values...
Photo images/stock/phone_photo (5).jpg already exists in the database. Using cache values...
Photo images/stock/image7.jpeg already exists in the database. Using cache values...
Photo images/stock/image5.jpeg already exists in the database. Using cache values...
Photo images/stock/phone_photo (10).jpg already exists in the database. Using cache values...
Photo images/stock/image2.jpg already exists in the database. Using cache values...
Photo images/stock/image4.jpeg already exists in the database. Using cache values...
Photo images/stock/cat_and_newspaper.jpg already exists in the database. Using cache values...
Photo images/stock/phone_photo (1).jpg already exists in the database. Using cache values...
Photo images/stock/image9.jpeg already exists in the database. Using cache values...
Photo images/stock/image

[PhotoInfo(photo_path='images/stock/image3.jpg', thumbnail_photo_path='NOT IMPLEMENTED', is_screenshot=False, tags=['barricade', 'barrier', 'car', 'construction site', 'construction worker', 'direct', 'dress', 'fireman', 'goggles', 'jumpsuit', 'man', 'pole', 'road', 'road sign', 'safety vest', 'sign', 'stand', 'street sign', 'traffic sign', 'wear', 'worker', 'yellow'], part_of_which_album=[], face_ids=[], metadata_fields={'SourceFile': 'images/stock/image3.jpg', 'FileName': 'image3.jpg', 'Directory': 'images/stock', 'FileSize': '64 KiB', 'FileAccessDate': '2024:01:24', 'FileAccessTime': '14:59:49', 'TimeZone': '+05:30', 'FileType': 'JPEG', 'Model': None, 'ISO': None, 'FNumber': None, 'ExposureTime': None, 'FocalLength': None, 'CreateDate': None, 'CreateTime': None, 'DateTimeOriginal': None, 'ShutterSpeedValue': None, 'GPSLatitudeRef': None, 'GPSLongitudeRef': None, 'Make': None, 'ImageWidth': 1024, 'ImageHeight': 682, 'Megapixels': 0.698, 'ShutterSpeed': None, 'GPSLatitude': None, 'GPS

In [7]:
# Add image to the index
from index import get_nodes_from_objects

if photo_info is not None:
    nodes = get_nodes_from_objects(photo_info, image_files=image_paths)
    index._add_nodes_to_index(nodes = nodes, index_struct=index.index_struct, show_progress=True)

Generating embeddings:   0%|          | 0/21 [00:00<?, ?it/s]

## Retriver

In [8]:
from llama_index.retrievers import VectorIndexAutoRetriever
from index import define_vector_info

vector_store_info = define_vector_info()

retriever = VectorIndexAutoRetriever(
    index,
    vector_store_info=vector_store_info,
    similarity_top_k=4,
    empty_query_top_k=10,  # if only metadata filters are specified, this is the limit
    verbose=True,
)

## Query

In [9]:
# from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from IPython.display import Image
from typing import List
from llama_index.schema import TextNode 

def display_response(nodes: List[TextNode]):
    """Display response."""
    for node in nodes:
        print(node.get_content(metadata_mode="all"))
        # img = Image.open(open(node.metadata["image_file"], 'rb'))
        display(Image(filename=node.metadata["image_metadata"]["SourceFile"], width=200))

In [18]:
nodes = retriever.retrieve(
    "Show me pictures I clicked at low ISO"
)
display_response(nodes)

Using query str: pictures clicked at low ISO
Using filters: [('image_metadata.ISO', '<=', 'low')]


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
